<a href="https://colab.research.google.com/github/mahmoudta74/FIDVR/blob/main/FIDVR_Khorasan_Test3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

from torch.utils.tensorboard import SummaryWriter
%load_ext tensorboard
from IPython.display import display, clear_output
import pandas as pd
import time
import json

from sklearn.preprocessing import MinMaxScaler
from pandas.plotting import register_matplotlib_converters
from tqdm.notebook import tqdm

import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torch.optim as optim

torch.set_printoptions(linewidth=120) #Disply Option for output
torch.set_grad_enabled(True)          # already on by default\


%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#93D30C", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 14, 10
register_matplotlib_converters()

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [2]:
!nvidia-smi

Mon May 31 15:17:44 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
print(torch.__version__)
print(torchvision.__version__)

1.8.1+cu101
0.9.1+cu101


In [5]:
# checking for GPU
torch.cuda.is_available()

True

In [6]:
df1 = pd.read_csv('/content/drive/MyDrive/dataset/Main_Dataset.csv')
df2 = pd.read_csv('/content/drive/MyDrive/dataset/result_rms1.csv')

data1 = df1.to_numpy()
data2 = df2.to_numpy()

In [7]:
train_data = data1[: , 123].reshape(-1, 1)
test_data  = data2[: , 123].reshape(-1, 1)

print(f'train_data size = {train_data.shape}\t test_data size = {test_data.shape}')

train_data size = (127051, 1)	 test_data size = (12130, 1)


In [8]:
def create_sequences(time, data, seq_length, num_run):
    xs = []
    ys = []
    t = time[0]
    j = 0

    for i in range(len(data) - (seq_length*num_run - num_run)):

        if t == 0:
            if j==0:
                x = data[j : (j + seq_length)]
                y = data[(j+ seq_length)]
                xs.append(x)
                ys.append(y)
                j = j+35
                t = time[j]
            else:
                x = data[j+1 : (j+1 + seq_length)]
                y = data[(j+1+ seq_length)]
                xs.append(x)
                ys.append(y)
                j = j+35
                t = time[j]


        else:
            x = data[j+1-seq_length : j+1]
            y = data[j+1]
            xs.append(x)
            ys.append(y)
            j = j+1
            t = time[j+1]

    return np.array(xs), np.array(ys)

In [9]:
seq_length = 35
time1 = data1[:,0]
time2 = data2[:,0]

X_train, y_train = create_sequences(time1, train_data, seq_length , num_run=1050)
X_test , y_test  = create_sequences(time2,  test_data , seq_length, num_run=105)

print(X_train.shape,'\t', y_train.shape)
print(X_test.shape,'\t', y_test.shape)

(91351, 35, 1) 	 (91351, 1)
(8560, 35, 1) 	 (8560, 1)


In [10]:
# You should build your custom dataset as below.
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, Data, target, Transform= None):
        self.data = Data
        self.label = target
        self.Transform = Transform
        
    def __getitem__(self, index):
        #r = self.data.iloc[index]
        label = torch.tensor(self.label[index], dtype= torch.float32)  
        sample = torch.tensor(self.data[index], dtype= torch.float32)
        if self.Transform:
            sample = self.Transform(sample)
        return (sample, label)

    def __len__(self):
        return self.data.shape[0]


In [11]:
from collections import OrderedDict
from collections import namedtuple
from itertools import product

In [12]:
class RunBuilder():
  @staticmethod       #Since the get_runs() method is static,
                      #we can call it using the class itself. We don't need an instance of the class.
  def get_runs(params):                

    Run = namedtuple('Run', params.keys()) #This line creates a new tuple subclass called Run that has named fields.
                                            # This Run class is used to encapsulate the data for each of our runs.
                                            # The field names of this class are set by the list of names passed to the 
                                            # constructor. First, we are passing the class name. Then,
                                            # we are passing the field names,
                                            # and in our case, we are passing the list of keys from our dictionary.

#Now that we have a class for our runs, we are ready to create some.
    runs = []
    for v in product(*params.values()):  #  v is a tuple like (0.01, 64, 'cuda')
      runs.append(Run(*v))               # *v is 
      
    return runs

In [13]:
class RunManager():
  
    def __init__(self):

        self.epoch_count = 0           # The number of epochs.
        self.epoch_loss = 0            # The running loss for an epoch.
        self.epoch_start_time = None   # The start time of the epoch.

        self.run_params = None         # This is the run definition in terms for the run parameters
        self.run_count = 0             # gives us the run number.
        self.run_start_time = None
        self.run_data = []             #  is a list we'll use to keep track of the parameter values and the
                                        # results of each epoch for each run and so we'll see that we
                                        # add a value to this list for each epoch

        self.network = None
        self.loader = None
        self.tb = None

#######################################################

    def begin_run(self, run, network, loader):

        self.run_start_time = time.time()

        self.run_params = run
        self.run_count += 1

        self.network = network
        self.loader = loader
        self.tb = SummaryWriter(comment=f'-{run}') # passing run as an comment argument to SummaryWriter

        #images, labels = next(iter(self.loader))
        #grid = torchvision.utils.make_grid(images)

        #self.tb.add_image('images', grid)
        #self.tb.add_graph(self.network, images)

############################################################

    def end_run(self):
        self.tb.close()
        self.epoch_count = 0

########################################################3##3

    def begin_epoch(self):
        self.epoch_start_time = time.time()

        self.epoch_count += 1
        self.epoch_loss = 0
        #self.epoch_num_correct = 0

##############################################################3

    def end_epoch(self):

        epoch_duration = time.time() - self.epoch_start_time
        run_duration = time.time() - self.run_start_time

        loss = self.epoch_loss #/ len(self.loader.dataset)
        #accuracy = self.epoch_num_correct / len(self.loader.dataset)

        self.tb.add_scalar('Loss', loss, self.epoch_count)
        #self.tb.add_scalar('Accuracy', accuracy, self.epoch_count)

        for name, param in self.network.named_parameters():
            self.tb.add_histogram(name, param, self.epoch_count)
            self.tb.add_histogram(f'{name}.grad', param.grad, self.epoch_count)


        results = OrderedDict()
        results["run"] = self.run_count
        results["epoch"] = self.epoch_count
        results['loss'] = loss
        #results["accuracy"] = accuracy
        results['epoch duration'] = epoch_duration
        results['run duration'] = run_duration

        for k,v in self.run_params._asdict().items(): results[k] = v

        self.run_data.append(results)

        df = pd.DataFrame.from_dict(self.run_data, orient='columns')

        clear_output(wait=True)     # clear current output
        display(df)                 # display new dataframe

################################################################

    def track_loss(self, loss, batch):
        self.epoch_loss += loss.item()
    '''
    def track_num_correct(self, preds, labels):
        self.epoch_num_correct += self._get_num_correct(preds, labels)

##############################################################

    @torch.no_grad()
    def _get_num_correct(self, preds, labels): # private method
        return preds.argmax(dim=1).eq(labels).sum().item()
        '''
################################################################

    def save(self, fileName):       # save run data to .csv .json format

        pd.DataFrame.from_dict(
            self.run_data, orient='columns').to_csv(f'{fileName}.csv')

        with open(f'{fileName}.json', 'w', encoding='utf-8') as f:
            json.dump(self.run_data, f, ensure_ascii=False, indent=4)

###############################################################

In [20]:
class Network(nn.Module):

    def __init__(self,seq_len, n_features, n_hidden, n_layers):
        super(Network, self).__init__()

        self.n_features = n_features
        self.n_hidden = n_hidden
        self.seq_len = seq_len
        self.n_layers = n_layers

        self.lstm = nn.LSTM(input_size= n_features, hidden_size= n_hidden,
                             num_layers = n_layers, batch_first = True,
                             dropout = 0.3
                             )
            

        self.linear = nn.Linear(in_features= n_hidden, out_features= 1)

    def reset_hidden_state(self):
        self.hidden = (
        torch.zeros(self.n_layers, self.seq_len, self.n_hidden),
        torch.zeros(self.n_layers, self.seq_len, self.n_hidden)
    )

    def forward(self, x):

        h0 = torch.randn(self.n_layers, x.size(0), self.n_hidden).to('cuda')  # (batch_size= x.size(0) )
        c0 = torch.randn(self.n_layers, x.size(0), self.n_hidden).to('cuda')
        output, (hn, cn) = self.lstm(x, (h0, c0))
        out = self.linear(hn[0])

        return out

In [23]:
train_set = CustomDataset(X_train, y_train, Transform= None)
test_set = CustomDataset(X_test, y_test, Transform= None)

In [24]:
params = OrderedDict(
    lr = [0.001, 0.0005]
    ,batch_size = [64]
    , num_workers = [0]        #,1,2 ,...
    , device = ['cuda']
)


m = RunManager()

for run in RunBuilder.get_runs(params):

    device = torch.device(run.device)
    network = Network(seq_len=35 ,n_features=1 , n_hidden=128 , n_layers=2).to(device)

    loader = DataLoader(train_set, batch_size = run.batch_size,
                                           num_workers = run.num_workers,
                                           drop_last=True,
                                           shuffle=False) 
    optimizer = optim.Adam(params= network.parameters(), lr= run.lr)
    loss_fn = torch.nn.MSELoss()

    m.begin_run(run, network, loader)

    for epoch in range(25):
        m.begin_epoch()
    
        for batch in loader:
            data = batch[0].to(device)
            labels = batch[1].to(device)
            preds = network(data)
            loss = loss_fn(preds, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            m.track_loss(loss, batch)

        m.end_epoch()
    m.end_run()

#m.save('results')

,run,epoch,loss,epoch duration,run duration,lr,batch_size,num_workers,device
0,1,1,16.899433,9.954635,9.956300,0.0010,64,0,cuda
1,1,2,2.172530,9.933815,20.005169,0.0010,64,0,cuda
2,1,3,1.418806,9.922020,30.036814,0.0010,64,0,cuda
3,1,4,3.079708,9.923062,40.076488,0.0010,64,0,cuda
4,1,5,0.645212,9.914428,50.102201,0.0010,64,0,cuda
5,1,6,0.375011,9.938058,60.154278,0.0010,64,0,cuda
6,1,7,0.899855,9.927430,70.205962,0.0010,64,0,cuda
7,1,8,0.647152,9.926431,80.250544,0.0010,64,0,cuda
8,1,9,0.550488,9.929826,90.301732,0.0010,64,0,cuda
9,1,10,0.276556,9.942662,100.375154,0.0010,64,0,cuda


In [ ]:
# Tensorboard results
tensorboard  --logdir=runs